In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('melb_data.csv')
print(data.shape)
print(data.columns)

(13580, 21)
Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [14]:
def score_dataSet(train_x,test_x,train_y,test_y):
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor()
    rfr.fit(train_x,train_y)
    print('模型拟合完成')
    from sklearn.metrics import mean_absolute_error
    print(mean_absolute_error(test_y,rfr.predict(test_x)))

In [11]:
y = data.Price
x = data.drop(['Price'],axis=1)

# 丢掉数据
## 先丢掉数据再进行数据划分

In [26]:
data_without_missing = x.dropna(axis=1)
num_data_without_missing = data_without_missing.select_dtypes(exclude=['object'])
print(num_data_without_missing.shape)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(num_data_without_missing,y,train_size=0.7,test_size=0.3, random_state=0)
score_dataSet(train_x,test_x,train_y,test_y)

(13580, 9)
模型拟合完成
189661.9399700774


## 先进行数据划分，再丢掉空数据

In [24]:
num_data_without_missing = x.select_dtypes(exclude=['object'])
col_with_missing = [t for t in num_data_without_missing.columns if num_data_without_missing[t].isnull().any()] # 对num_data_without_missing[t]列的每个数据进行isnull判断，但会一些列Ture/False
print(col_with_missing)
print(num_data_without_missing.shape)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(num_data_without_missing,y,train_size=0.7,test_size=0.3, random_state=0)
train_x = train_x.drop(col_with_missing,axis=1)
test_x  = test_x.drop(col_with_missing,axis=1)
score_dataSet(train_x,test_x,train_y,test_y)

['Car', 'BuildingArea', 'YearBuilt']
(13580, 12)
模型拟合完成
189679.82756738435


## 结论
上述两个方案，由于存在随机划分数据的因素，所以求得的数据不完全相同，但是最后误差大致上都相同，跟期望相符


# 进行数据填充

In [33]:
from sklearn.preprocessing import Imputer
im = Imputer()
num_data = x.select_dtypes(exclude=['object'])

## 直接进行填充

In [34]:
x_imputer = im.fit_transform(num_data)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x_imputer,y,train_size=0.7,test_size=0.3, random_state=0)
score_dataSet(train_x,test_x,train_y,test_y)

模型拟合完成
182739.37931715642


## 数据先复制，再对为空的数据，新建列，再对新列进行数据填充

In [38]:
new_num_data = num_data.copy()
print('填充前:',new_num_data.shape)
cols_with_missing = (col for col in new_num_data.columns 
                                 if new_num_data[col].isnull().any())
for col in cols_with_missing:
    new_num_data[col + '_was_missing'] = new_num_data[col].isnull()

# Imputation
imputed_new_num_data = im.fit_transform(new_num_data)
print('填充后:',imputed_new_num_data.shape)
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(imputed_new_num_data,y,train_size=0.7,test_size=0.3, random_state=0)
score_dataSet(train_x,test_x,train_y,test_y)

填充前: (13580, 12)
填充后: (13580, 15)
模型拟合完成
182816.76692848958


## 结论
上面的两种填充方法误差是相差不多的。但是仅在这个数据集上，在研究其它问题中，第二种方法往往效果好一点

## 注意事项
### what
对比作者给出的代码
发现，作者是先将数据分割，然后在对train，test分别进行填充。而且，针对两种数据集的填充方式不同

imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)  #使用fit_transform 函数

imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)  #使用transform函数

二者的功能都是对数据进行某种统一处理（比如标准化~N(0,1)，将数据缩放(映射)到某个固定区间，归一化，正则化等） fit_transform(partData)对部分数据先拟合fit，找到该part的整体指标，如均值、方差、最大值最小值等等（根据具体转换的目的），然后对该partData进行转换transform，从而实现数据的标准化、归一化等等

根据对之前部分fit的整体指标，对剩余的数据（restData）使用同样的均值、方差、最大最小值等指标进行转换transform(restData)，从而保证part、rest处理方式相同。 必须先用fit_transform(partData)，之后再transform(restData) 如果直接transform(partData)，程序会报错 如果fit_transfrom(partData)后，使用fit_transform(restData)而不用transform(restData)，虽然也能归一化，但是两个结果不是在同一个“标准”下的，具有明显差异。

### 结论
fit_transfrom(train_data)

transfrom(test_data) #保证训练集和测试集在同一标准下

而不能使用
fit_transfrom(train_data)

fit_transfrom(test_data)

如果使用下面方法，效果会更好

在数据划分前
先fit_transform(x)
划分后
transform(train_x)
transform(test_x)

In [45]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0,1))
train = [[10,100,10,1],[20,500,2,0.1]]
test = [[5,80,4,0.5],[50,500,2,0.9]]
print(mms.fit_transform(train))
print(mms.transform(test))
print(mms.fit_transform(test)) # 可以使用MinMaxScaler 看出fit_transform和transform的差别


[[0. 0. 1. 1.]
 [1. 1. 0. 0.]]
[[-0.5        -0.05        0.25        0.44444444]
 [ 4.          1.          0.          0.88888889]]
[[0. 0. 1. 0.]
 [1. 1. 0. 1.]]
